In [ ]:
!git clone https://github.com/rinnakk/japanese-pretrained-models

In [ ]:
%cd /content/japanese-pretrained-models

In [ ]:
!pip install -r requirements.txt

In [26]:
import torch
from transformers import T5Tokenizer, RobertaForMaskedLM

# load tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

# load model
model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
model = model.eval()

print('#'*40)
print('答えさせたい内容を「〜、何。」と聞いてみてください。')
print('例；世界一うるさい人と言ったら、何。')
print('※最後の「。」を忘れずに。')
print('#'*40)

# original text
text = input()

# prepend [CLS]
text = "[CLS]" + text

if '何' in text:
  # tokenize
  tokens = tokenizer.tokenize(text)
  # mask a token
  masked_idx = -2
  tokens[masked_idx] = tokenizer.mask_token
  print('#'*40)
  print('#以下の[MASK]の部分を考えます。')
  print(tokens)  
  print('#'*40)
# convert to ids
  token_ids = tokenizer.convert_tokens_to_ids(tokens)

  # convert to tensor
  token_tensor = torch.tensor([token_ids])

  # get the top 10 predictions of the masked token

  with torch.no_grad():
      outputs = model(token_tensor)
      predictions = outputs[0][0, masked_idx].topk(10)

  print('#'*40)
  print('#思いついた答えは・・・')
  for i, index_t in enumerate(predictions.indices):
      index = index_t.item()
      token = tokenizer.convert_ids_to_tokens([index])[0]
      print(i, token)
  print('#以上です。')
  print('#'*40)

else:
  print('何か聞いてくれないとわからないです。')  


########################################
答えさせたい内容を「〜、何。」と聞いてみてください。
例；世界一うるさい人と言ったら、何。
※最後の「。」を忘れずに。
########################################
世界一の天才と言ったら、何。
########################################
#以下の[MASK]の部分を考えます。
['[CLS]', '▁', '世界一', 'の', '天才', 'と言った', 'ら', '、', '[MASK]', '。']
########################################
########################################
#思いついた答えは・・・
0 アインシュタイン
1 あなた
2 <unk>
3 チャップリン
4 ドラえもん
5 わからない
6 織田信長
7 誰か
8 サッカー選手
9 天才
#以上です。
########################################
